Titre....

In [ ]:
#Cellule pour lire stop_times.txt stocké sur Google Drive car trop volumineux pour être stocké sur le dépôt GitHub
import os, gdown

#lien : https://drive.google.com/file/d/1qLf7bU6Z2rVIvQ2y0dZYmiA9CGluwCMS/view?usp=sharing
file_id = "1qLf7bU6Z2rVIvQ2y0dZYmiA9CGluwCMS"
url = f"https://drive.google.com/uc?id={file_id}"
os.makedirs("data", exist_ok=True)
out_path = "data/gtfs/stop_times.txt"
gdown.download(url, out_path, quiet=False)  # gère aussi les gros fichiers >100 Mo

stop_times = pd.read_csv("data/gtfs/stop_times.txt") #heure d'arrivée de trip_id à stop_id
